## CS 736: Assignment - Image Denoising with MRFs

##### Kartik Nair

In [ ]:

import numpy as np
%pip install pymatreader seabo
from pymatreader import read_mat
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

data = Path('../data')

### Question 1 -  Denoising a Phantom Magnetic Resonance Image.

Implement a maximum-a-posteriori Bayesian image-denoising algorithm that uses a suitable noise model (i.i.d. Gaussian or independent Rician) coupled with a MRF prior that uses a 4-neighbor neighborhood system (each pixel has 4 neighbors: left, right, up, down; the neighborhood wraps around at image boundaries) that has cliques of size no more than 2.

Use gradient ascent (or descent) optimization with dynamic step size; implementing a fixed step size isn’t acceptable. Ensure that the value of the objective function (i.e., the log posterior or its negative) at each iteration increases (or decreases if using gradient descent). Use the noisy image as the initial solution.

Use 3 different MRF priors where the potential functions $V(x_i , x_j ) := g(x_i - x_j)$ underlying the MRF penalize the difference between the neighboring voxel values $x_i , x_j$ as follows (see class notes for details). ( You may rely on the `circshift()` function in Matlab  or `numpy.roll()` when computing differences between every pixel in the image and its neighbors.)

Introduce a parameter $\alpha \in [0, 1]$ to control the weighting between the prior (weight $\alpha$) and the likelihood (weight $1 - \alpha$).

Specifically, implement the following functionality as part of the denoising algorithm:


1. A suitable noise model. You don’t need the noise level because that parameter can be absorbed in $1 - \alpha$ that you’ll tune manually (Tuning $\alpha$ essentially manipulates the noise level, in case of the likelihood. So we can ignore the noise level $\sigma$ when tuning $\alpha$ manually. Use $\sigma = 1$).
2. MRF prior: Quadratic function: $$g_1(u) := |u|^2$$
3. MRF prior: Discontinuity-adaptive Huber function: $$g_2(u) := 0.5|u|^2,$$ when $|u| \leq \gamma$ and $$g(u) := \gamma|u| - 0.5\gamma^2,$$ when $|u| > \gamma$, where $0 < \gamma < \infty$ is a constant.
4. MRF prior: Discontinuity-adaptive function: $$g_3(u) := \gamma|u| - \gamma^  2 log(1 + |u|/\gamma),$$ where $0 < \gamma < \infty$ is a constant.


For each MRF prior, manually tune the parameters $\alpha$ and $\gamma$ (where applicable) to denoising the noisy image in order to achieve the least possible relative root-mean-squared error (RRMSE).The RRMSE for 2 images $A$ and $B$ is defined as : 
$$RRMSE(A, B) = \frac{\sqrt{\sum_p (|A(p)| - |B(p)|)^2}}{\sqrt{\sum_p|A(p)|^2}} ,$$
where the summation is over all pixels $p$. Always use the noiseless image as $A$.

In [ ]:
pmridata = read_mat(data/'assignmentImageDenoisingPhantom.mat')
pmriimageNoiseless = pmridata['imageNoiseless']
pmriimageNoisy = pmridata['imageNoisy']
pmriimageNoisy = pmriimageNoisy.flatten()

#### Maximum A Posteriori Estimation

Let $X$ be the original image, and $X^*$ the noisy image taken.

We will then take $X$ such that it is the likeliest given $X^*$, i.e $\argmax_X P(X|X^*)$

$$P(X|X^*) = P(X^*|X) \frac{P(X)}{P(X^*)}$$
$P(X^*|X)$ will be based on the weighted noise model - taken standard Normal here.
$P(X)$, the prior, will be based on the MRF model $\frac{1}{Z}e^{-\sum V_c(X_c)}$ weighted

So, we have
$$P(X|X^*) = k \mathcal{N}(X-X^*,\mu=0,\sigma=1)^{1-\alpha}\cdot \left(e^{-\sum V_c(X_c)}\right)^{\alpha},$$
where $k$ absorbs he other constant factors.

$$\log\left(P(X|X^*)\right) = (1-\alpha)\log{\mathcal{N}(X-X^*,\mu=0,\sigma=1)} -\alpha\sum V_c(X_c) + \log k$$
From this we get our loss function - 
$$\mathcal{L} = - \left[(1-\alpha)\left(-\frac{1}{2}\lVert X-X^*\rVert^2\right) - \alpha\frac{1}{2}\left(\sum_{X_i \in X} \sum_{Y_i \in N(X_i)} g(X_i- Y_i)\right)\right]$$
Where $X = \left(X_1,X_2,\cdots, X_d\right)$, $g$ is the potential function for pairs in a clique in the image grid.


$$\mathcal{L} = \frac{1}{2}(1-\alpha)\lVert X-X^*\rVert^2 + \frac{1}{2}\alpha\sum_{X_i \in X} \sum_{Y_i \in N(X_i)} g(X_i- Y_i) $$

We shall optimise with gradient descent.

The gradient is 
$$\frac{\partial\mathcal{L}}{\partial X} = \frac{(1-\alpha)}{2} \frac{\partial \lVert X-X^*\rVert^2}{\partial X} +\frac{\alpha}{2} \sum_{X_i \in X} \sum_{Y_i \in N(X_i)} \frac{\partial g(X_i- Y_i)}
{\partial X}$$

Let us revisit $h$ as defined in class.

For $g(\eta_i) = H(|\eta_i|^2)$, $h(\eta_i)$ is defined as $$h(\eta_i) = \frac{\partial H(|\eta_i|^2)}{\partial |\eta_i|^2}$$
Thus $$\frac{\partial g(\eta_i)}{\partial\eta_i} = 2\eta_i h(\eta_i)$$

Let us rewrite $h$ for a vectorised input - 4 types of neighbours are possible for a pixel {up , down, left, right}. The corresponding neighbours can be found for the entire image by circularly shifting the image vector. We denote by N(X) the set of 4 vectors formed by these 4 shifts.

Thus
$$\frac{\partial\mathcal{L}}{\partial X} = (1-\alpha)(X-X^*) + \alpha \sum_{Y \in N(X)} (X-Y)\odot h(X-Y)$$

The gradient descent step is:
$$x \mapsto \left. x - \tau\frac{\partial{\mathcal{L}}}{\partial X}\right|_x$$

For dynamically changing the learning rate $\tau$ we will be implementing the backtracking line search algorithm.

In [ ]:
## The MRF losses and corresponding gradeints
g1 = lambda u : u**2
h1 = lambda u: np.ones(u)
uh1 = lambda u : u

g2 = lambda gam : (lambda u : 0.5*u**2*(abs(u)<gam) + (abs(u)>=gam)*(gam*abs(u) - 0.5*gam**2) )
h2 = lambda gam : (lambda u: 0.5*(abs(u)<gam) + gam*(abs(u)>=gam)/(2*abs(u)))
uh2 = lambda gam : (lambda u: u*(0.5*(abs(u)<gam) + gam*(abs(u)>=gam)/(2*abs(u))))

g3 = lambda gam : (lambda u: gam*abs(u) - gam**2*np.log(1+abs(u)/gam))
h3 = lambda gam : (lambda u: gam/(2*(abs(u)+gam)))
uh3 = lambda gam : (lambda u: gam*u/(2*(abs(u)+gam)))

In [ ]:
#Gradient delL / del X

LossGradient = lambda Xnoise: (lambda utimeshfunc : (lambda alpha : (lambda X: (1-alpha)*(X-Xnoise)+alpha*(utimeshfunc(X - np.roll(X,1))+utimeshfunc(X - np.roll(X,-1))+utimeshfunc(X - np.roll(X,256))+utimeshfunc(X - np.roll(X,-256))))))

Loss = lambda Xnoise: (lambda gfunc : (lambda alpha : (lambda X: (1-alpha)*np.dot(X-Xnoise,X-Xnoise)+alpha*(gfunc(X - np.roll(X,1)).sum()+gfunc(X - np.roll(X,-1)).sum()+gfunc(X - np.roll(X,256)).sum()+gfunc(X - np.roll(X,-256)).sum()))))

RRMSE = lambda Noiseless: (lambda Noisy: np.linalg.norm(Noiseless-Noisy)/np.linalg.norm(Noiseless))

In [ ]:
def GradDescOpt(Xnoisy, uthfunc,gfunc, alpha): # change this to have dynamic lr
    lr =1
    GDU = LossGradient(Xnoisy)(uthfunc)(alpha)
    LF = Loss(Xnoisy)(gfunc)(alpha)
    X_map = Xnoisy
    obj = np.array([])
    for _ in range(1000):
        L,GL = LF(X_map),GDU(X_map)
        obj.append(L)
        
        X_map -= lr*GL
    return X_map, obj

def tune(Xnoisy,uthf,gf,gamma = False):
    vals = {}
    rrmse = RRMSE(Xnoisy)
    #alpha lies between 0 and 1
    # gamma lies between 0 and inf, but mainly along ___
    if not gamma:
        for i in range(10): # placeholder
            alpha = 0.5
            vals[(alpha,)] = GradDescOpt(Xnoisy,uthf,gf,alpha)
    else :
        for i in range(100):
            alpha = 0.5
            gamma = 1
            vals[(alpha,gamma)] = GradDescOpt(Xnoisy,uthf(gamma),gf(gamma),alpha)

    return vals

Report the following:
1. Report the RRMSE between the noisy and noiseless images.

2. Report the optimal values of the parameters and the corresponding RRMSEs for each of the 3 denoising algorithms. For each optimal parameter value reported (for each of the 3 denoising algorithms), give evidence of the optimality of the reported values by reporting the RRMSE values for two nearby parameter values (around the optimal) at plus/minus $20\%$ of the optimal value. That is, if $a^{*}$, $b^{*}$ are the optimal parameter values, then report: $a^{*}, b^{*}$, $RRMSE(a^{*}, b^{*})$, $RRMSE(1.2a^{*}, b^{*})$, $RRMSE(0.8a^{*}, b^{*})$,$RRMSE(a^{*}, 1.2b^{*})$, $RRMSE(a^{*}, 0.8b^{*})$.
(Tip: the optimal values for $\alpha$ might be very close to extreme limits of the allowed range. Beaware of that possibility.).

3. Show the following 5 images (at each pixel, show the magnitude of the pixel value; use a jet colormap) in the report using exactly the same colormap 
- Noiseless image, 
- Noisy image, 
- Image denoised using quadratic prior $g_1(\cdot)$ and optimal parameter tuning, 
- Image denoised using Huber prior $g_1(\cdot)$ and optimal parameter tuning, 
- Image denoised using discontinuity-adaptive prior $g_3(\cdot)$ and optimal parameter tuning.

4. Show the plots of the objective-function values (vertical axis) versus iteration (horizontal axis) corresponding to each of the 3 denoised results above.

### Question 2 - Denoising a Magnetic Resonance Image of the Brain.

Repeat the same task for the brain MRI image in the `data` folder.

### Question 3 - Designing a Bayesian Denoising Formulation for a RGB Microscopy Image.

1. Design a suitable MRF prior model, assuming statistical dependencies within a spatial neighborhood as well as across the three colors channels, i.e., red, green, and blue.
2. Design a suitable noise model.
3. Design a suitable Bayesian denoising formulation.
